In [ ]:
import numpy as np 
import pandas as pd

In [ ]:
%%time
df_full = pd.read_csv('/kaggle/input/us-used-cars-dataset/used_cars_data.csv', nrows=5e5) #/kaggle/input/us-used-cars-dataset/
from sklearn.model_selection import train_test_split
#df_train, df_test = train_test_split(df_full, test_size=0.25, random_state=1, shuffle=True)

In [ ]:
%%time
#,'transmission_display','wheel_system'
cols = ['height','length','width','year','listed_date','is_new','body_type','city_fuel_economy','daysonmarket','engine_cylinders','engine_displacement','front_legroom','fuel_tank_volume','fuel_type','has_accidents','highway_fuel_economy','horsepower','major_options','make_name','maximum_seating','mileage','model_name','owner_count','seller_rating','transmission_display','wheel_system','torque','city']
colY = ['price']
df_full = df_full[cols+colY].copy()

In [ ]:
%%time
train, test = train_test_split(df_full, test_size=0.25, random_state=1, shuffle=True)
train, test = train.copy(), test.copy()

In [ ]:
%%time
samp = train

samp.owner_count = np.where(samp.is_new,0,np.where(samp.owner_count.isna(),2,samp.owner_count))
samp.has_accidents = pd.to_numeric(np.where(samp.is_new,0,np.where(samp.has_accidents.isna(),1,samp.has_accidents)))
samp.fillna({'body_type':'Sedan'}, inplace=True)
samp['age'] = np.where(samp.is_new,0,samp.listed_date.str[:4].astype(int)-samp.year)

test.owner_count = np.where(test.is_new,0,np.where(test.owner_count.isna(),2,test.owner_count))
test.has_accidents = pd.to_numeric(np.where(test.is_new,0,np.where(test.has_accidents.isna(),1,test.has_accidents)))
test.fillna({'body_type':'Sedan'}, inplace=True)
test['age'] = np.where(test.is_new,0,test.listed_date.str[:4].astype(int)-test.year)

In [ ]:
pd.DataFrame({'nan_perc_train':samp.isna().sum()/len(samp)*100, 'nan_perc_test':test.isna().sum()/len(test)*100, \
              'type':samp.dtypes}).sort_values(by='nan_perc_train', ascending=False)

In [ ]:
%%time
from sklearn.preprocessing import OrdinalEncoder
cols_ = ['body_type','engine_cylinders','fuel_type','make_name','model_name','transmission_display','wheel_system','city']
samp[cols_] = samp[cols_].fillna('nan')
test[cols_] = test[cols_].fillna('nan')

encoder = OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1)
samp[cols_] = encoder.fit_transform(samp[cols_])
test[cols_] = encoder.transform(test[cols_])


In [ ]:
%%time
# обработка данных
cast_dict = {' in':['height','length','width','front_legroom'], ' gal':['fuel_tank_volume'], ' seats':['maximum_seating']}
intcols = ['maximum_seating']

for s,l in cast_dict.items():
    for col in l:
        samp[col] = np.where(samp[col].str[-len(s):]==s,samp[col].str[:-len(s)],np.nan)
        samp[col] = pd.to_numeric(samp[col], errors='coerce')
        
        test[col] = np.where(test[col].str[-len(s):]==s,test[col].str[:-len(s)],np.nan)
        test[col] = pd.to_numeric(test[col], errors='coerce')

In [ ]:
col = 'torque'
samp[col] = pd.to_numeric(samp[col].str[:3], errors='coerce')
test[col] = pd.to_numeric(test[col].str[:3], errors='coerce')

In [ ]:
mcols = ['height','length','width','front_legroom','fuel_tank_volume','torque','city_fuel_economy','highway_fuel_economy','mileage']
samp.loc[:,mcols] = samp[mcols].fillna(samp[mcols].mean())
test.loc[:,mcols] = test[mcols].fillna(samp[mcols].mean())

In [ ]:
df_train = samp
# gradientboosting
from sklearn.metrics import mean_absolute_error
from numpy import mean, std
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score, RepeatedKFold

In [ ]:
%%time
colX = ['height','length','width','is_new','body_type','daysonmarket','engine_cylinders','front_legroom','fuel_tank_volume','fuel_type','has_accidents','highway_fuel_economy','horsepower','maximum_seating','mileage','owner_count']
colX = ['height','length','width','year','is_new','body_type','city_fuel_economy','daysonmarket','engine_cylinders','engine_displacement','front_legroom','fuel_tank_volume','fuel_type','has_accidents','highway_fuel_economy','horsepower','maximum_seating','mileage','owner_count','seller_rating','transmission_display','wheel_system','torque','make_name','model_name','city']
# colX = ['height','length','year','mileage','city_fuel_economy','daysonmarket']
gbr_train = GradientBoostingRegressor(max_depth=15, n_estimators=50)
X_train, y_train = samp[colX].fillna(samp[colX].mean()),samp[colY].values.ravel()
X_test,  y_test  = test[colX].fillna(samp[colX].mean()),test[colY].values.ravel()
gbr_train.fit(X_train, y_train)
y_predict = gbr_train.predict(X_test)
mae = mean_absolute_error(y_test,y_predict)
print(f"model MAE: {mae:.0f}")

In [ ]:
display(pd.DataFrame({'col':colX,'importance':gbr_train.feature_importances_*100}).sort_values('importance',ascending=False))

In [ ]:
pd.DataFrame({'nan_perc_train':samp[colX].isna().sum()/len(samp)*100, 'nan_perc_test':test[colX].isna().sum()/len(test)*100, \
              'type':samp[colX].dtypes}).sort_values(by='nan_perc_train', ascending=False)

In [ ]:
# model MAE: 3060
# Wall time: 24min 46s

# without 'transmission_display','wheel_system'
# model MAE: 2323
# Wall time: 3min 11s

# all
# model MAE: 2343
# Wall time: 3min 21s

In [ ]:

# df_train = samp
# y = df_train[colY]

# # simple models

# name = "1. all_mean"
# mprice = y.mean()[0]
# mae = mean_absolute_error(y,[mprice]*len(y))
# print(f"{name} MAE: {mae:.0f}; mean={mprice:.0f}")

# name = "2. make_mean"
# gcols = ['make_name']
# mprice = df_train.groupby(gcols)["price"].transform(mean)
# mae = mean_absolute_error(y,mprice)
# print(f"{name} MAE: {mae:.0f}")

# name = "3. model_mean"
# gcols = ['make_name','model_name']
# mprice = df_train.groupby(gcols)["price"].transform(mean)
# mae = mean_absolute_error(y,mprice)
# print(f"{name} MAE: {mae:.0f}")

# name = "4. model_age_mean"
# gcols = ['make_name','model_name','age']
# mprice = df_train.groupby(gcols)["price"].transform(mean)
# mae = mean_absolute_error(y,mprice)
# print(f"{name} MAE: {mae:.0f}")
